In [25]:
import concurrent.futures
import pandas as pd

def parse_node_coord(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, x, y = map(int, line.split())
        data.append({'Index': index, 'X': x, 'Y': y})
    return pd.DataFrame(data)

def parse_item_section(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, profit, weight, assigned_node = map(int, line.split())
        data.append({'Index': index, 'Profit': profit, 'Weight': weight, 'Assigned_Node': assigned_node})
    return pd.DataFrame(data)

def parse_file_parallel(lines, dimension):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_node_coord = executor.submit(parse_node_coord, lines, 10, dimension)
        future_profit_matrix = executor.submit(parse_item_section, lines, 10 + dimension + 1, dimension)

    node_coord = future_node_coord.result()
    profit_matrix = future_profit_matrix.result()

    return node_coord, profit_matrix


# Example usage:

with open('a280-n279.txt', 'r') as file:
    lines = file.readlines()

variables = parse_variables(lines)
node_coord, profit_matrix = parse_file_parallel(lines, variables['dimension'])

print("Variables:", variables)
print("Node Coord:", node_coord)
print("Profit Matrix:", profit_matrix)


Variables: {'dimension': 280, 'num_items': 279, 'capacity': 25936, 'min_speed': 0.1, 'max_speed': 1.0, 'renting_ratio': 5.61, 'edge_weight_type': 'CEIL_2D'}
Node Coord:      Index    X    Y
0        1  288  149
1        2  288  129
2        3  270  133
3        4  256  141
4        5  256  157
..     ...  ...  ...
275    276  236  145
276    277  246  141
277    278  252  125
278    279  260  129
279    280  280  133

[280 rows x 3 columns]
Profit Matrix:      Index  Profit  Weight  Assigned_Node
0        1     101       1              2
1        2     202       2              3
2        3     404       4              4
3        4     202       2              5
4        5     996     896              6
..     ...     ...     ...            ...
274    275     786     686            276
275    276    1572    1372            277
276    277     786     686            278
277    278     566     466            279
278    279     456     356            280

[279 rows x 4 columns]


In [26]:
variables

{'dimension': 280,
 'num_items': 279,
 'capacity': 25936,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 5.61,
 'edge_weight_type': 'CEIL_2D'}

In [28]:
node_coord

,Index,X,Y
0,1,288,149
1,2,288,129
2,3,270,133
3,4,256,141
4,5,256,157
...,...,...,...
275,276,236,145
276,277,246,141
277,278,252,125
278,279,260,129


In [29]:
node_coord

,Index,X,Y
0,1,288,149
1,2,288,129
2,3,270,133
3,4,256,141
4,5,256,157
...,...,...,...
275,276,236,145
276,277,246,141
277,278,252,125
278,279,260,129


In [31]:
profit_matrix

,Index,Profit,Weight,Assigned_Node
0,1,101,1,2
1,2,202,2,3
2,3,404,4,4
3,4,202,2,5
4,5,996,896,6
...,...,...,...,...
274,275,786,686,276
275,276,1572,1372,277
276,277,786,686,278
277,278,566,466,279


In [32]:
import numpy as np

def calculate_distance_matrix(node_coord):
    coordinates = node_coord[['X', 'Y']].values
    x, y = np.meshgrid(coordinates[:, 0], coordinates[:, 1])

    distance_matrix = np.sqrt((x - x.T)**2 + (y - y.T)**2)
    
    return distance_matrix


In [33]:
calculate_distance_matrix(node_coord)

array([[ 0.        , 20.        , 24.08318916, ..., 43.26661531,
        34.40930107, 17.88854382],
       [20.        ,  0.        , 18.43908891, ..., 36.22154055,
        28.        ,  8.94427191],
       [24.08318916, 18.43908891,  0.        , ..., 19.6977156 ,
        10.77032961, 10.        ],
       ...,
       [43.26661531, 36.22154055, 19.6977156 , ...,  0.        ,
         8.94427191, 29.12043956],
       [34.40930107, 28.        , 10.77032961, ...,  8.94427191,
         0.        , 20.39607805],
       [17.88854382,  8.94427191, 10.        , ..., 29.12043956,
        20.39607805,  0.        ]])